In [2]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules


In [82]:
df = pd.read_csv('Groceries_dataset.csv')

In [83]:
df

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [84]:
df[df['Member_number']== 1808] 

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
4355,1808,04-02-2015,long life bakery product
9090,1808,29-11-2014,meat
11488,1808,15-12-2014,sugar
16149,1808,21-07-2015,rolls/buns
20504,1808,04-02-2015,semi-finished bread
25239,1808,29-11-2014,whole milk
27637,1808,15-12-2014,citrus fruit
36088,1808,21-07-2015,candy
38731,1808,15-12-2014,napkins


In [85]:
agrupando = df[df['Member_number'] == 1808].groupby('Date')['itemDescription'].sum()
agrupando

Date
04-02-2015    long life bakery productsemi-finished bread
15-12-2014                       sugarcitrus fruitnapkins
21-07-2015                  tropical fruitrolls/bunscandy
29-11-2014                                 meatwhole milk
Name: itemDescription, dtype: object

In [87]:
transacoes_agrupadas = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index()
transacoes_agrupadas

,Member_number,Date,itemDescription
0,1000,15-03-2015,"[sausage, whole milk, semi-finished bread, yog..."
1,1000,24-06-2014,"[whole milk, pastry, salty snack]"
2,1000,24-07-2015,"[canned beer, misc. beverages]"
3,1000,25-11-2015,"[sausage, hygiene articles]"
4,1000,27-05-2015,"[soda, pickled vegetables]"
...,...,...,...
14958,4999,24-01-2015,"[tropical fruit, berries, other vegetables, yo..."
14959,4999,26-12-2015,"[bottled water, herbs]"
14960,5000,09-03-2014,"[fruit/vegetable juice, onions]"
14961,5000,10-02-2015,"[soda, root vegetables, semi-finished bread]"


In [89]:
# Instanciar o codificador de transações
te = TransactionEncoder()

In [90]:
# Transformar os dados para o formato adequado
te_ary = te.fit(transacoes_agrupadas['itemDescription']).transform(transacoes_agrupadas['itemDescription'])
df_te = pd.DataFrame(te_ary, columns=te.columns_)


In [91]:
df_te

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [105]:
# Aplicar o algoritmo FP-Growth
frequent_itemsets = fpgrowth(df_te, min_support=0.005, use_colnames=True)

# Exibir os conjuntos de itens frequentes
print(frequent_itemsets)

      support                     itemsets
0    0.157923                 (whole milk)
1    0.085879                     (yogurt)
2    0.060349                    (sausage)
3    0.009490        (semi-finished bread)
4    0.051728                     (pastry)
..        ...                          ...
121  0.007151   (bottled beer, whole milk)
122  0.005280  (domestic eggs, whole milk)
123  0.005614     (newspapers, whole milk)
124  0.007151   (citrus fruit, whole milk)
125  0.005012           (pork, whole milk)

[126 rows x 2 columns]


In [110]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.005)


In [111]:
soda_rules = rules[rules['antecedents'].apply(lambda x: 'soda' in x)]


In [112]:
soda_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(soda),(yogurt),0.097106,0.085879,0.005814,0.059876,0.697219,-0.002525,0.972342,-0.324769
15,(soda),(sausage),0.097106,0.060349,0.005948,0.061253,1.014975,0.000088,1.000963,0.016341
22,(soda),(whole milk),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636,-0.260917
25,(soda),(other vegetables),0.097106,0.122101,0.009691,0.099794,0.817302,-0.002166,0.975219,-0.198448
27,(soda),(rolls/buns),0.097106,0.110005,0.008087,0.083276,0.757022,-0.002596,0.970843,-0.262257
41,(soda),(tropical fruit),0.097106,0.067767,0.005413,0.055747,0.822622,-0.001167,0.987270,-0.192778
51,(soda),(root vegetables),0.097106,0.069572,0.005280,0.054370,0.781501,-0.001476,0.983925,-0.236442


In [115]:
soda_rules_sorted = soda_rules.sort_values(by='confidence', ascending=False)


In [116]:
soda_rules_sorted

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
22,(soda),(whole milk),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636,-0.260917
25,(soda),(other vegetables),0.097106,0.122101,0.009691,0.099794,0.817302,-0.002166,0.975219,-0.198448
27,(soda),(rolls/buns),0.097106,0.110005,0.008087,0.083276,0.757022,-0.002596,0.970843,-0.262257
15,(soda),(sausage),0.097106,0.060349,0.005948,0.061253,1.014975,0.000088,1.000963,0.016341
3,(soda),(yogurt),0.097106,0.085879,0.005814,0.059876,0.697219,-0.002525,0.972342,-0.324769
41,(soda),(tropical fruit),0.097106,0.067767,0.005413,0.055747,0.822622,-0.001167,0.987270,-0.192778
51,(soda),(root vegetables),0.097106,0.069572,0.005280,0.054370,0.781501,-0.001476,0.983925,-0.236442
